In [ ]:
pip install openai

In [ ]:
import openai
import pandas as pd


# Read the Excel file
file_path = 'food.csv'
data = pd.read_csv(file_path)

# Select the 'fdc_id' and 'description' columns from the first Excel file
selected_columns1 = data[['fdc_id', 'description']]

# Read the second Excel file
file_path2 = 'input_food.csv'
data2 = pd.read_csv(file_path2)

# Select the 'fdc_id' and 'sr_description' columns from the second Excel file
selected_columns2 = data2[['fdc_id', 'sr_description']]

# Rename the 'sr_description' column to 'description'
selected_columns2 = selected_columns2.rename(columns={'sr_description': 'description'})

# Concatenate the two DataFrames
combined_data = pd.concat([selected_columns1, selected_columns2], ignore_index=True)




# Save the combined data to a new Excel file
output_file_path = 'combined_excel_file.xlsx'

combined_data.to_excel(output_file_path, index=False)

# Read the 'food_nutrient' Excel file
food_nutrient_file_path = 'food_nutrient.csv'
food_nutrient_data = pd.read_csv(food_nutrient_file_path)

# Read the 'nutrient' Excel file
nutrient_file_path = 'nutrient.csv'
nutrient_data = pd.read_csv(nutrient_file_path)

# Merge 'food_nutrient_data' with 'nutrient_data' on the nutrient_id/id
merged_data = food_nutrient_data.merge(nutrient_data, left_on='nutrient_id', right_on='id')

# Group the data by 'fdc_id' and aggregate the nutrient names, units, and amounts as lists
grouped_data = merged_data.groupby('fdc_id').agg(
    nutrients=pd.NamedAgg(column='name', aggfunc=list),
    units=pd.NamedAgg(column='unit_name', aggfunc=list),
    amounts=pd.NamedAgg(column='amount', aggfunc=list)
).reset_index()

# Merge 'combined_data' with 'grouped_data' on the 'fdc_id' column
final_data = combined_data.merge(grouped_data, on='fdc_id')

# Save the final data to a new Excel file
final_output_file_path = 'final_excel_file.xlsx'

final_data.to_excel(final_output_file_path, index=False)

In [ ]:
# Read the final Excel file
final_excel_file_path = 'final_excel_file.xlsx'
final_data = pd.read_excel(final_excel_file_path)

# Create a new DataFrame with two columns: 'input_text' and 'target_text'
new_data = pd.DataFrame(columns=['input_text', 'target_text'])

# Assign the 'description' column to the 'input_text' column
new_data['input_text'] = final_data['description']

# Concatenate the 'nutrients', 'units', and 'amounts' columns into a single string for each row
# and assign it to the 'target_text' column
new_data['target_text'] = final_data['nutrients'].astype(str) + ', ' + \
                          final_data['units'].astype(str) + ', ' + \
                          final_data['amounts'].astype(str)

# Save the new DataFrame to a CSV file
new_csv_file_path = 'input_target_data.csv'
new_data.to_csv(new_csv_file_path, index=False)

In [ ]:
#We then use Open AI CLI from windows command to fine tune the model

For cli:

- Open Windows shell/command. Ensure openai cli is installed and in the correct path as your data folder. 

- Export open ai key using env or directly. You can get a API KEY on OPEN AI site after registering.

	
	python export OPENAI_API_KEY= <OPENAI KEY>

(refer to the documentation for more info but find your code mgiht need some playing around with)


- This cli tool checks the data to see if it needs further cleaning.
    

	openai tools fine_tunes.prepare_data -f <Your data file full path>


- Once the client tool check completes, either it okays the current file or creates a new one. If a new one is created ensure that name is reflecte in below code. This code starts the fine tune process.

 	set OPENAI_API_KEY= <Key> && openai api fine_tunes.create -t  C:\Users\tm568\OneDrive\Desktop\Codes\GPT_Python_Test\latest_filtered_input_target_data_prepared_prepared.jsonl



- To check status use
    
    openai api fine_tunes.follow -i <Model id>


-  To see the list of fine_tunes, their status, which files you used to train them. 
    
    openai api fine_tunes.list

    
    
- Documentation: 
   
    https://platform.openai.com/docs/guides/fine-tuning

